#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
#!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

#Review: Solution to Participation Question
minimize f(x) = 4x^4 - 3x^3 - 9x + 20

In [ ]:
#initialize a "Concrete Model"
model = ConcreteModel()

#initialize DVs
model.x = Var(domain = Reals) #You can google "pyomo domain sets" to see several domain examples

#define the objective
model.Objective = Objective(expr = 4*model.x**4 - 3*model.x**3 - 9*model.x + 20, sense = minimize)

#(Optional) You can use model.pprint() to see what you've done so far
model.pprint()

1 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*x**4 - 3*x**3 - 9*x + 20

2 Declarations: x Objective


In [ ]:
#solve model
opt = SolverFactory('ipopt', executable='/content/ipopt')

results = opt.solve(model, tee = True) #setting tee = False hides the diagnostic outputs

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [ ]:
#print relevant values
print("x* = ", model.x()) #alternatively you can use value(model.x)
print("obj* = ", model.Objective()) #alternatively you can use value(model.Objective)

x* =  1.0616077811644107
obj* =  11.936814677331192


##Repeating with Objective as a Function


In [ ]:
#define the objective function
def obj_function(x):
  return 4*x**4 - 3*x**3 - 9*x + 20

#initialize a "Concrete Model"
model2 = ConcreteModel()

#initialize DVs
model2.x = Var(domain = Reals) #You can google "pyomo domain sets" to see several domain examples

#define the objective using the function defined earlier
model2.Objective = Objective(expr = obj_function(model2.x), sense = minimize)

#(Optional) You can use model.pprint() to see what you've done so far
model2.pprint()

1 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*x**4 - 3*x**3 - 9*x + 20

2 Declarations: x Objective


In [ ]:
#solve model & print relevant values
opt = SolverFactory('ipopt', executable='/content/ipopt')

results = opt.solve(model2, tee = False) #setting tee = False hides the diagnostic outputs

print("x* = ", model2.x()) #alternatively you can use value(model.x)
print("obj* = ", model2.Objective()) #alternatively you can use value(model.Objective)

x* =  1.0616077811644107
obj* =  11.936814677331192


#Demo: Optimizing Existing Models (Adbudget)
We created the below model in the previous module. Use ipopt in Pyomo to find the optimal price. 

In [ ]:
def ad_profit(unit_cost = 25,
              sales_price = 40,
              seasonality_factor = [.9, 1.1, .8, 1.2],
              salesforce_cost = [8000, 8000, 9000, 9000],
              overhead_rate = .15,
              k = 35,
              b = 3000,
              ad_spend = [10000, 10000, 10000, 10000],
              num_quarters = 4):
  #Gross profit calculations
  sales_quantity = [k*seasonality_factor[i]*((b + ad_spend[i])**.5) for i in range(num_quarters)]
  cogs = [unit_cost*sales_quantity[i] for i in range(num_quarters)]
  sales_rev = [sales_price*sales_quantity[i] for i in range(num_quarters)]
  gross_profit = [sales_rev[i] - cogs[i] for i in range(num_quarters)]
  total_gross_profit = sum(gross_profit)
  #operating expense calculation
  overhead = [overhead_rate*sales_rev[i] for i in range(num_quarters)]
  operating_expenses = [overhead[i] + salesforce_cost[i] + ad_spend[i] for i in range(num_quarters)]
  total_operating_expenses = sum(operating_expenses)
  #operating profit
  operating_profit = total_gross_profit - total_operating_expenses
  return operating_profit

In [ ]:
num_quarters = 4

#initialize a "Concrete Model"
model3 = ConcreteModel()

#initialize DVs
model3.ad_spend = Var(range(num_quarters), domain = NonNegativeReals)

#define the objective
model3.operating_profit = Objective(expr = ad_profit(ad_spend = model3.ad_spend), sense = maximize)

#(Optional) You can use model.pprint() to see what you've done so far
model3.pprint()

1 Set Declarations
    ad_spend_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {0, 1, 2, 3}

1 Var Declarations
    ad_spend : Size=4, Index=ad_spend_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    operating_profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*(31.5*(3000 + ad_spend[0])**0.5) - 25*(31.5*(3000 + ad_spend[0])**0.5) + 40*(38.5*(3000 + ad_spend[1])**0.5) - 25*(38.5*(3000 + ad_spend[1])**0.5) + 40*(28.0*(3000 + ad_spend[2])**0.5) - 25*(28.0*(3000 + ad_spend[2])**0.5) + 40*(42.0*(3000 + ad_spen

In [ ]:
#solve model & print relevant values
opt = SolverFactory('ipopt', executable='/content/ipopt')

results = opt.solve(model3, tee = False) #setting tee = False hides the diagnostic outputs

In [ ]:
for i in range(num_quarters):
  print(f"optimal adspend in Q{i+1} = {model3.ad_spend[i]()}")
print(f"optimal operating profit is {model3.operating_profit()}")

optimal adspend in Q1 = 17093.06250000487
optimal adspend in Q2 = 27015.562500004056
optimal adspend in Q3 = 12876.00000000539
optimal adspend in Q4 = 32721.00000000369
optimal operating profit is 79705.62500000003


#Your Turn A: Practice writing an optimizing problem
A rectangular box must have volume at least 500 in^3. Find the shape that has the smallest "mailing length" (the sum of the three edge lengths). What is this minimum "mailing length?" (Use ipopt since this is a nonlinear problem). I've started some code for you.

In [ ]:
#initialize a "Concrete Model"
model = ConcreteModel()

#initialize DVs
model.x1 = Var(domain = NonNegativeReals)
model.x2 = Var(domain = NonNegativeReals)
model.x3 = Var(domain = NonNegativeReals)

#define the objective
model.Objective = Objective(expr = model.x1 + model.x2 + model.x3 , sense = minimize)

#define the constraint
model.Constraint = Constraint(expr = model.x1*model.x2*model.x3 >= 500)

#(Optional) You can use model.pprint() to see what you've done so far
model.pprint()

In [ ]:
#solve model & print relevant values
opt = SolverFactory('ipopt', executable='/content/ipopt')

results = opt.solve(model, tee = True) #setting tee = False hides the diagnostic outputs

In [ ]:
#print results
print("x =", model.x1())
print("y =", model.x2())
print("z =", model.x3())
print("Obj = ", model.Objective())
print("Constraint = ", model.Constraint())


x = 7.937005234219425
y = 7.937005234219425
z = 7.937005234219425
Obj =  23.811015702658274
Constraint =  499.9999951578263


#Your Turn B: Using Existing Functions (office building)
Below, I've included a completed "office building" model as a function. Use Pyomo to solve for the price per square foot in each year that maximizes the total earnings after tax. Use ipopt (since this is a nonlinear problem).

In [43]:
def office_earnings(total_sqft = 180000,
           m = -0.05,
           b = 1.5,
           op_expense_per_sqft = 1.20,
           heating_surcharge_per_sqft = .2,
           op_exp_annual_growth = .12,
           annual_mortgage = 1500000,
           tax_rate = .34,
           price_per_sqft = [15, 15, 15, 15, 15],
           num_years = 5):
  #rev calc
  perc_occ = [m*price_per_sqft[i] + b for i in range(num_years)]
  sqft_occ = [perc_occ[i]*total_sqft for i in range(num_years)]
  revenue = [sqft_occ[i]*price_per_sqft[i] for i in range(num_years)]
  #operating expense calculations
  base_op_cost_as_percY1 = [(1+op_exp_annual_growth)**i for i in range(num_years)] #note that range(num_years) = range(5) = [0,1,2,3,4] and (1+op_exp_annual_growth)**0 = 1.
  base_op_cost = [op_expense_per_sqft*total_sqft*base_op_cost_as_percY1[i] for i in range(num_years)]
  heating_surcharge = [perc_occ[i]*base_op_cost[i]*heating_surcharge_per_sqft for i in range(num_years)]
  mortgage = [annual_mortgage for i in range(num_years)]
  operating_costs = [base_op_cost[i] + heating_surcharge[i] + mortgage[i] for i in range(num_years)]
  #before and after-tax earnings
  ebt = [revenue[i] - operating_costs[i] for i in range(num_years)]
  taxes = [ebt[i]*tax_rate for i in range(num_years)]
  earnings_after_tax = [ebt[i] - taxes[i] for i in range(num_years)]
  total_earnings_after_tax = sum(earnings_after_tax)
  return total_earnings_after_tax

In [ ]:
num_years = 5
#initialize a "Concrete Model"
model6 = ConcreteModel()

#initialize DVs
model6.pricesqft = Var(range(num_years), domain = NonNegativeReals)

#define the objective
model6.Objective = Objective(expr = office_earnings(price_per_sqft=model6.pricesqft) , sense = maximize)

#(Optional) You can use model.pprint() to see what you've done so far
model6.pprint()

In [45]:
#solve model & print relevant values
opt = SolverFactory('ipopt', executable='/content/ipopt')

 #setting tee = False hides the diagnostic outputs
results = opt.solve(model6, tee = False)

In [46]:
# results

print("Obj =", model6.Objective())

Obj = 691696.8342059832
